In [1]:
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
import plotly.express as px
import pandas as pd
import numpy as np

In [69]:
beer = pd.read_csv(r"beer_train.csv")
beer

,beer/ABV,beer/brewerId,review/appearance,review/aroma,review/overall,review/palate,review/taste,user/birthdayRaw,"beer_name_""The Wind Cried Mari..."" Scottish Heather Ale",beer_name_'Pooya Porter,...,yday_356,yday_357,yday_358,yday_359,yday_360,yday_361,yday_362,yday_363,yday_364,yday_365
0,5.00,14338,4.0,4.0,4.0,4.0,4.0,0.0,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,11.00,395,4.0,3.5,3.5,3.5,3.0,0.0,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.70,365,3.5,4.0,3.5,3.5,3.5,0.0,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.40,1,3.0,3.0,2.5,3.0,3.0,1976.0,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.40,1417,4.0,3.0,3.0,3.5,2.5,0.0,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37060,5.50,3268,4.0,3.5,3.5,3.5,3.5,0.0,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37061,8.50,1199,4.5,4.0,3.5,4.5,4.5,0.0,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37062,4.75,394,4.0,3.5,4.0,4.5,4.0,0.0,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37063,11.20,1199,4.0,4.0,4.0,5.0,5.0,0.0,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
beer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37065 entries, 0 to 37064
Columns: 2466 entries, beer/ABV to yday_365
dtypes: bool(1781), float64(675), int64(10)
memory usage: 256.7 MB


#### Convert Bool to Int

In [71]:
columns = beer.columns
for column in columns:
    if beer[column].dtype == bool:
        beer[column] = beer[column].astype(np.int8)
beer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37065 entries, 0 to 37064
Columns: 2466 entries, beer/ABV to yday_365
dtypes: float64(675), int64(10), int8(1781)
memory usage: 256.7 MB


In [72]:
beer

,beer/ABV,beer/brewerId,review/appearance,review/aroma,review/overall,review/palate,review/taste,user/birthdayRaw,"beer_name_""The Wind Cried Mari..."" Scottish Heather Ale",beer_name_'Pooya Porter,...,yday_356,yday_357,yday_358,yday_359,yday_360,yday_361,yday_362,yday_363,yday_364,yday_365
0,5.00,14338,4.0,4.0,4.0,4.0,4.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,11.00,395,4.0,3.5,3.5,3.5,3.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.70,365,3.5,4.0,3.5,3.5,3.5,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.40,1,3.0,3.0,2.5,3.0,3.0,1976.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.40,1417,4.0,3.0,3.0,3.5,2.5,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37060,5.50,3268,4.0,3.5,3.5,3.5,3.5,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37061,8.50,1199,4.5,4.0,3.5,4.5,4.5,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37062,4.75,394,4.0,3.5,4.0,4.5,4.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37063,11.20,1199,4.0,4.0,4.0,5.0,5.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
beer.shape

(37065, 2466)

In [74]:
beer.drop(columns=beer.columns[beer.sum() == 0], inplace=True)

In [75]:
beer.shape

(37065, 2449)

In [156]:
x = beer.drop(columns=["review/overall"])
y = beer["review/overall"]

In [77]:
x.shape, y.shape

((37065, 2448), (37065,))

In [78]:
pd.DataFrame(x).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37065 entries, 0 to 37064
Columns: 2448 entries, beer/ABV to yday_365
dtypes: float64(674), int64(9), int8(1765)
memory usage: 255.5 MB


In [157]:
sc = StandardScaler()
x = sc.fit_transform(x)

In [80]:
# To prevent Data Leaakage
kFold = KFold(n_splits=5, shuffle=True, random_state=250)

## Randomized search CV

### Lasso Regression

In [81]:
lr = Lasso()

In [82]:
# Regularization strength
lasso_params = {'alpha': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0]}

In [83]:
rcv = RandomizedSearchCV(estimator=lr, param_distributions=lasso_params,
                         scoring="r2", refit=False, cv=kFold, verbose=2, n_jobs=-1, n_iter=6, random_state=250)

In [84]:
rcv.fit(X=x, y=y)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


RandomizedSearchCV(cv=KFold(n_splits=5, random_state=250, shuffle=True),
                   estimator=Lasso(), n_iter=6, n_jobs=-1,
                   param_distributions={'alpha': [0.0001, 0.001, 0.01, 0.1, 1.0,
                                                  10.0]},
                   random_state=250, scoring='r2', verbose=2)

In [85]:
rcv.best_score_

0.6419993420070252

In [86]:
rcv.best_params_

{'alpha': 0.001}

In [96]:
l = Lasso(alpha=0.001)
l.fit(x, y)

Lasso(alpha=0.001)

In [112]:
l.coef_

array([-0.04626659, -0.00187185,  0.03826855, ..., -0.        ,
       -0.00192868,  0.        ])

In [99]:
l.l1_ratio

1.0

In [131]:
coefs = pd.DataFrame({
    "Columns": beer.drop(columns=["review/overall"]).columns,
    "coef": l.coef_
})

In [132]:
coefs

,Columns,coef
0,beer/ABV,-0.046267
1,beer/brewerId,-0.001872
2,review/appearance,0.038269
3,review/aroma,0.072068
4,review/palate,0.156643
...,...,...
2443,yday_361,0.000000
2444,yday_362,-0.000000
2445,yday_363,-0.000000
2446,yday_364,-0.001929


### The Columns that are Not Important according to Lasso Regression

In [140]:
coefs[coefs["coef"] == 0]["Columns"].values

(1010,)

#### There are 1010 Columns that are not Important

In [141]:
coefs[coefs["coef"] == 0]["Columns"].values.shape

(1010,)

### Important Columns:

In [142]:
coefs[coefs["coef"] != 0]["Columns"].values

array(['beer/ABV', 'beer/brewerId', 'review/appearance', ..., 'yday_357',
       'yday_358', 'yday_364'], dtype=object)

#### There are 1438 Columns that are important

In [139]:
coefs[coefs["coef"] != 0]["Columns"].values.shape

(1438,)

### Dropping columns that are not Important

In [147]:
coefs[coefs["coef"] == 0]["Columns"].index

Index([   7,   13,   17,   18,   19,   20,   22,   23,   32,   36,
       ...
       2431, 2433, 2434, 2437, 2441, 2442, 2443, 2444, 2445, 2447],
      dtype='int64', length=1010)

In [160]:
x = np.delete(x, coefs[coefs["coef"] == 0]["Columns"].index, axis=1)

In [161]:
x.shape

(37065, 1438)

In [162]:
x

array([[-1.04357291,  2.21020359,  0.15346778, ..., -0.04737441,
        -0.04679889, -0.05886731],
       [ 1.54806257, -0.51507322,  0.15346778, ..., -0.04737441,
        -0.04679889, -0.05886731],
       [-1.17315469, -0.52093697, -0.72057698, ..., -0.04737441,
        -0.04679889, -0.05886731],
       ...,
       [-1.15155772, -0.51526868,  0.15346778, ..., -0.04737441,
        -0.04679889, -0.05886731],
       [ 1.63445042, -0.35792465,  0.15346778, ..., -0.04737441,
        -0.04679889, -0.05886731],
       [ 0.46821445, -0.35792465,  0.15346778, ..., -0.04737441,
        -0.04679889, -0.05886731]])

### Ridge Regression

In [89]:
r = Ridge()

In [91]:
# Regularization strength
ridge_params = {'alpha': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0]}

In [92]:
rcv = RandomizedSearchCV(estimator=r, param_distributions=ridge_params,
                         scoring="r2", refit=False, cv=kFold, verbose=2, n_jobs=-1, n_iter=6, random_state=250)

In [93]:
rcv.fit(X=x, y=y)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


RandomizedSearchCV(cv=KFold(n_splits=5, random_state=250, shuffle=True),
                   estimator=Ridge(), n_iter=6, n_jobs=-1,
                   param_distributions={'alpha': [0.0001, 0.001, 0.01, 0.1, 1.0,
                                                  10.0]},
                   random_state=250, scoring='r2', verbose=2)

In [94]:
rcv.best_score_

0.6300041457732097

In [95]:
rcv.best_params_

{'alpha': 10.0}

### ElasticNet Regression

In [163]:
e = ElasticNet()

In [164]:
elastic_net_params = {
    # Experiment with different L1 vs. L2 regularization ratios
    'l1_ratio': [0.1, 0.5, 0.9],
    # Explore a range of regularization strengths
    'alpha': [0.0001, 0.001, 0.01, 0.1],
}

In [165]:
rcv = RandomizedSearchCV(estimator=e, param_distributions=elastic_net_params, scoring="r2",
                         refit=False, cv=kFold, verbose=2, n_jobs=-1, n_iter=10, random_state=250)

In [166]:
rcv.fit(x, y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=KFold(n_splits=5, random_state=250, shuffle=True),
                   estimator=ElasticNet(), n_jobs=-1,
                   param_distributions={'alpha': [0.0001, 0.001, 0.01, 0.1],
                                        'l1_ratio': [0.1, 0.5, 0.9]},
                   random_state=250, scoring='r2', verbose=2)

In [168]:
rcv.best_params_

{'l1_ratio': 0.1, 'alpha': 0.01}

In [169]:
rcv.best_score_

0.6464788906878456

### DecisionTreeRegressor

In [172]:
dt = DecisionTreeRegressor()

In [175]:
decision_tree_params = {
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 5, 10],
    'max_features': ['auto', 'log2', 'sqrt'],
    'splitter': ['best', 'random'],
}

In [176]:
rcv = RandomizedSearchCV(estimator=dt, param_distributions=decision_tree_params,
                         scoring="r2", refit=False, cv=kFold, verbose=2, n_jobs=-1, n_iter=10, random_state=250)

In [177]:
rcv.fit(x, y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=KFold(n_splits=5, random_state=250, shuffle=True),
                   estimator=DecisionTreeRegressor(), n_jobs=-1,
                   param_distributions={'max_depth': [3, 5, 10, None],
                                        'max_features': ['auto', 'log2',
                                                         'sqrt'],
                                        'min_samples_leaf': [1, 5, 10],
                                        'min_samples_split': [2, 5, 10, 20],
                                        'splitter': ['best', 'random']},
                   random_state=250, refit=False, scoring='r2', verbose=2)

In [179]:
rcv.best_params_

{'splitter': 'random',
 'min_samples_split': 20,
 'min_samples_leaf': 5,
 'max_features': 'auto',
 'max_depth': 10}

In [178]:
rcv.best_score_

0.600444545533174

### RandomForestRegressor

In [181]:
rf = RandomForestRegressor(n_jobs=-1)

In [187]:
random_forest_params = {
    'n_estimators': [100, 200, 500],  # Number of trees in the forest
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 5, 10],
    'max_features': ['auto', 'log2', 'sqrt'],
}

In [188]:
rcv = RandomizedSearchCV(estimator=rf, param_distributions=random_forest_params,
                         scoring="r2", refit=False, cv=kFold, verbose=2, n_jobs=-1, n_iter=10, random_state=250)

In [189]:
rcv.fit(x, y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=KFold(n_splits=5, random_state=250, shuffle=True),
                   estimator=RandomForestRegressor(n_jobs=-1), n_jobs=-1,
                   param_distributions={'max_depth': [3, 5, 10, None],
                                        'max_features': ['auto', 'log2',
                                                         'sqrt'],
                                        'min_samples_leaf': [1, 5, 10],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 500]},
                   random_state=250, refit=False, scoring='r2', verbose=2)

In [190]:
rcv.best_params_

{'n_estimators': 100,
 'min_samples_split': 10,
 'min_samples_leaf': 10,
 'max_features': 'auto',
 'max_depth': None}

In [191]:
rcv.best_score_

0.6411712788635772

### KNeighborsRegressor

In [210]:
knn_regressor = KNeighborsRegressor(n_jobs=-1)

In [214]:
knn_params = {
    'n_neighbors': [5, 10, 20],
    'metric': ['euclidean', 'manhattan'],
}

In [215]:
rcv = RandomizedSearchCV(estimator=knn_regressor, param_distributions=knn_params,
                         scoring="r2", refit=False, cv=kFold, verbose=2, n_jobs=-1, n_iter=10, random_state=250)

In [216]:
rcv.fit(x, y)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


0it [00:00, ?it/s]

In [217]:
rcv.best_params_

{'n_neighbors': 20, 'metric': 'manhattan'}

In [218]:
rcv.best_score_

0.34832993219211683

## Visualize The Models

In [2]:
algos = np.array(["Lasso", "Ridge", "ElasticNet", "DecisionTreeRegressor",
                 "RandomForestRegressor", "KNeighborsRegressor"])

r2_score = np.array([0.6419993420070252, 0.6300041457732097, 0.6464788906878456,
                    0.600444545533174, 0.6411712788635772, 0.34832993219211683])

In [3]:
px.bar(x=algos, y=r2_score, title="Algorithms vs Accuracy (r2 score)").update_xaxes(
    title="Algorithms").update_yaxes(title="r2 score")

In [229]:
accuracyTable = pd.DataFrame({"Algorithm": algos, "Accuracy": r2_score})
accuracyTable

,Algorithm,Accuracy
0,Lasso,0.641999
1,Ridge,0.630004
2,ElasticNet,0.646479
3,DecisionTreeRegressor,0.600445
4,RandomForestRegressor,0.641171
5,KNeighborsRegressor,0.348330


## Model with Maximum Accuracy

In [236]:
accuracyTable[accuracyTable["Accuracy"] == accuracyTable["Accuracy"].max()]

,Algorithm,Accuracy
2,ElasticNet,0.646479


# The model `ElasticNet` with Maximum Accuracy of 64.64%

## Best Parameters of the Model
- '`l1_ratio`' : 0.1
- '`alpha`' : 0.01

## Hyperparameter Tuning: Balancing Efficiency and Accuracy

I used Randomized Search to tune the model's settings (hyperparameters). This approach is faster than other methods, which is important because time is limited. While it might not find the absolute best settings, it's a good starting point.

**Next Steps:**

* If we have more time and resources, we could try a different method like Grid Search for even better accuracy.
* Depending on the model, other techniques might be explored for even faster optimization.

By using different methods, we can find the best settings for the model while keeping in mind time constraints.
